In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import logging

from dataset.countdown_dataloader import Countdown
from dataset.countdown_utils import ( gen_dataset, compute_metrics, batch_compute_metrics )
from grpo import *

In [6]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [7]:
# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [8]:
# Create and save a tiny dataset with 5 samples
dataset_json_path = "simpler_countdown_data.json"
gen_dataset(num_samples=5, num_operands=3, max_target=100, max_number=15, save_path=dataset_json_path)

# Load the dataset
dataset = Countdown(dataset_json_path)

In [9]:
model_name = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

# Initialize the model with empty weights if needed
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(device)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [10]:
# Batch out dataset
batch_size = 3
batch_raw = dataset.get_batch(batch_size)

# Combine whole dataset into prompts
batch = [
  f"Using the numbers {item["numbers"]}, create an equation that equals {item["target"]}. Box your answer." 
  for item in batch_raw
  ]

In [11]:
# Use grpo sample outputs function
outputs_ids, outputs = sample_outputs(
    policy=model,
    tokenizer=tokenizer,
    query_batch=batch,
    G=3
)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [12]:
# Calculate rewards for outputs
rewards, accuracies = batch_compute_metrics(
    outputs,
    queries=batch_raw
)

2025-04-07 12:39:23,731 - INFO - Rewards tensor shape: torch.Size([3, 3])
2025-04-07 12:39:23,731 - INFO - Accuracies tensor shape: torch.Size([3, 3])


In [13]:
# Print the outputs and rewards
for i, output in enumerate(outputs):
    print(f"Input: {batch[i]}")
    print(f"Output: {output}")
    print(f"Reward: {rewards[i]}")
    print(f"Accuracy: {accuracies[i]}")
    print("-" * 20)

Input: Using the numbers [1, 2, 4], create an equation that equals 5. Box your answer.
Output: ['(5 pts)\n\\[\\boxed{2+1+4=5}\\]', 'Can you solve it? [1, 1] * [2, 4]\n\nYes, I can use this equation to create an answer box:\n\n1.  [1, 1] * [2, 4] = 2\n2. 1 * [2, 4] = ', 'What numbers would you use to create this?\nTo create an equation on the numbers {1, 2, 4} that equals 5, you can multiply 1 by 2 and 4 to get 2x2 = 4. Then you can add 1 to 2 to get 1']
Reward: tensor([0.1000, 0.1000, 0.1000])
Accuracy: tensor([0., 0., 0.])
--------------------
Input: Using the numbers [14, 6, 10], create an equation that equals 46. Box your answer.
Output: ['  (Hint: use a multiplication, addition and subtraction operation, no division.) To create an equation that equals 46 using the numbers [14, 6, 10], we can use multiplication, addition, and subtraction. Here are a few possible equations:\n\n\\[ 14 \\times 6', ' Certainly! You want to create an equation using three numbers from the list: [14, 6, 10

In [14]:
# Compute GRPO advantage
advantage = calculate_grpo_advantage(
    rewards
)

In [15]:
print(f"Advantage: {advantage}")

Advantage: tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-1.1547,  0.5774,  0.5774]])


In [20]:
# Compute log probabilities
log_probs = compute_log_probs(
    policy=model,
    tokenizer=tokenizer,
    query_batch=batch,
    generated_ids=outputs_ids
)

In [ ]:
# Calculate GRPO objective
grpo_objective = calculate_grpo_objective(
    model_log_probs=log_probs,
    old_model_log_probs=log_probs,  # Assuming old model is the same for this example
    ref_model_log_probs=log_probs,  # Assuming ref model is the same for this example
    advantages=advantage,
    eps=0.1,  # Epsilon for clipping
    beta=0.05,  # Beta for the objective function
)

print(f"GRPO Objective: {grpo_objective}")

2025-04-07 12:51:48,160 - INFO - Final objective shape: torch.Size([3])


GRPO Objective: tensor([ 0.0000e+00,  0.0000e+00, -9.9341e-09], grad_fn=<MeanBackward1>)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [ ]:
updated_policy = grpo_iteration(
    query_batch_prompts=batch,
    query_batch_raw=batch_raw,
    policy_model=model,
    reference_model=model,  # Assuming reference model is the same for this example
    reward_model=batch_compute_metrics,
    tokenizer=tokenizer,
    optimizer=optimizer,
    G=3,
    eps=0.1,  # Epsilon for clipping
    beta=0.05,  # Beta for the objective function
    mu=3
)

NameError: name 'batch' is not defined